## Chunking E2E Sample

This notebook explores some of the evaluation metrics that can be used to measure the affectiveness of the retrieval of chunked content. This notebook is a complete end-to-end solution that engests pre-embedded content into Azure AI search and evaluates the results of search on a QA pair dataset.

This notebook plans to outline two evaluation metrics:

- RougeL: ROUGE-L (Recall-Oriented Understudy for Gisting Evaluation - Longest Common Subsequence) is a metric used in the field of Natural Language Processing for evaluating the quality of summaries by comparing them to reference summaries. 
- In-top: Checks if the expected source is contained in any of the chunks. If the source is found in atleast 1 of the retrieved chunks, the in-top is 1. If the source is not found in the retrieved chunks, the in-top is 0.

#### Importing the necessary packages

In [1]:
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import RawVectorQuery
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import ResourceNotFoundError
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    SearchIndex,
    SemanticConfiguration,
    PrioritizedFields,
    SemanticField,
    SearchField,
    SemanticSettings,
    VectorSearch,
    HnswVectorSearchAlgorithmConfiguration,
    HnswParameters,
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchProfile,
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    VectorSearch,
    HnswVectorSearchAlgorithmConfiguration,
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    SearchIndex,
    SemanticConfiguration,
    PrioritizedFields,
    SemanticField,
    SearchField,
    SemanticSettings,
    VectorSearch,
    HnswVectorSearchAlgorithmConfiguration,
    HnswParameters,
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchProfile
)

import dataclasses
from dataclasses import dataclass
from dotenv import dotenv_values
import json
from openai import AzureOpenAI
from openai.resources import Embeddings
import pandas as pd
from rouge_score import rouge_scorer
import time

_rouge_scorer = rouge_scorer.RougeScorer(rouge_types=["rougeL"], use_stemmer=True)

#### Setting up Configs

This notebook expects several environment variables (which are outlined below). The environment variables are used to connect to **Azure AI Search** and **Azure OpenAI**.

The required environment variables are the following:
- **AIS_ENDPOINT** [Required]: The Azure AI Search endpoint.
- **AIS_API_VERSION** [Required]: The Azure AI Search version (ex: `2023-11-01`).
- **AIS_KEY** [Required]: The Azure AI Search key.
- **AOAI_ENDPOINT** [Required]: The Azure OpenAI endpoint.
- **AOAI_API_VERSION** [Required]: The Azure OpenAI version (ex: `2023-09-01-preview`).
- **AZURE_OPENAI_KEY** [Required]: The Azure OpenAI key.
- **AOAI_EMBEDDING_DEPLOYED_MODEL** [Required]: The Azure OpenAI embedding model.

In [2]:
env_values = dotenv_values()

class Constants(object):
    INDEX_NAME = "evaluation-index"
    QA_DATASET_PATH = "../../../code_samples/data/e2e_samples/chunking/ground_truth/qa_dataset.csv"
    EMBEDDING_DATA_PATH = "../../../code_samples/data/e2e_samples/chunking/embeddings.json"
    OUTPUT_FILE_PATH = "./output/chunk_results.json"

class AzureAiSearchConfig(object):
    _API_ENDPOINT_KEY = "AIS_ENDPOINT"
    _API_VERSION_KEY = "AIS_API_VERSION"
    _API_KEY_KEY = "AIS_KEY"

    _REQUIRED_KEYS = [
        _API_ENDPOINT_KEY,
        _API_VERSION_KEY,
        _API_KEY_KEY
    ]

    api_endpoint: str
    api_version: str
    api_key: str

    def __init__(self, env_values: dict[str, any]):
        _api_endpoint = env_values.get(AzureAiSearchConfig._API_ENDPOINT_KEY)
        _api_version = env_values.get(AzureAiSearchConfig._API_VERSION_KEY)
        _api_key = env_values.get(AzureAiSearchConfig._API_KEY_KEY)

        if (not _api_endpoint or not _api_version or not _api_key):
            raise ValueError(f"The following environment variables are required: {', '.join(AzureAiSearchConfig._REQUIRED_KEYS)}")

        self.api_endpoint = _api_endpoint
        self.api_version = _api_version
        self.api_key = _api_key

class AzureOpenAiConfig(object):
    _API_ENDPOINT_KEY = "AOAI_ENDPOINT"
    _API_VERSION_KEY = "AOAI_API_VERSION"
    _API_KEY_KEY = "AZURE_OPENAI_KEY"
    _DEPLOYMENT_MODEL_KEY = "AOAI_EMBEDDING_DEPLOYED_MODEL"

    _REQUIRED_KEYS = [
        _API_ENDPOINT_KEY,
        _API_VERSION_KEY,
        _API_KEY_KEY,
        _DEPLOYMENT_MODEL_KEY
    ]

    api_endpoint: str
    api_version: str
    api_key: str
    deployment_model: str

    def __init__(self, env_values: dict[str, any]):
        _api_endpoint = env_values.get(AzureOpenAiConfig._API_ENDPOINT_KEY)
        _api_version = env_values.get(AzureOpenAiConfig._API_VERSION_KEY)
        _api_key = env_values.get(AzureOpenAiConfig._API_KEY_KEY)
        _deployment_model = env_values.get(AzureOpenAiConfig._DEPLOYMENT_MODEL_KEY)

        if (not _api_endpoint or not _api_version or not _api_key):
            raise ValueError(f"The following environment variables are required: {', '.join(AzureOpenAiConfig._REQUIRED_KEYS)}")

        self.api_endpoint = _api_endpoint
        self.api_version = _api_version
        self.api_key = _api_key
        self.deployment_model = _deployment_model


azure_openai_config = AzureOpenAiConfig(env_values)
azure_ai_search_config = AzureAiSearchConfig(env_values)

#### Defining Helper Classes

In [3]:
# azure search fields
_ID_FIELD = "id"
_CONTENT_FIELD = "chunk_content"
_VECTOR_FIELD = "chunk_content_vector"
_METADATA_FIELD = "metadata"
_SOURCE_METADATA_FIELD = "source"

_SELECT_FIELDS = [
    _ID_FIELD,
    _CONTENT_FIELD,
    _VECTOR_FIELD,
    _METADATA_FIELD,
]

# index config fields
_DEFAULT_SEMANTIC_CONFIG_NAME = "default"
_HNSW_ALGORITHM_CONFIG_NAME = "hnsw_config"
_VECTOR_SEARCH_PROFILE_NAME = "hnsw_profile"

# hnsw configs
_M = 4
_EF_CONSTRUCTION = 400
_EF_SEARCH = 500
_METRIC = "cosine"

@dataclass
class SearchResult:
    id: str
    content: str
    score: float
    source: str


class _Embedding(object):
    _embeddings: Embeddings
    _embedding_model: str

    def __init__(self, embeddings: Embeddings, embedding_model: str):
        self._embeddings = embeddings
        self._embedding_model = embedding_model

    def embed(self, text: str):
        return self._embeddings.create(input = [text], model=self._embedding_model).data[0].embedding

    @classmethod
    def from_config(cls, config: AzureOpenAiConfig):
        client = AzureOpenAI(
            api_key = config.api_key,  
            api_version = config.api_version,
            azure_endpoint = config.api_endpoint
        )
        embeddings = client.embeddings
        return cls(embeddings, config.deployment_model)


class _SearchClient(object):
    _BATCH_SIZE = 1000

    search_client: SearchClient

    def __init__(self, search_client: SearchClient):
        self._search_client = search_client

    def search(
        self,
        query: str,
        embedding_vector: list[float],
        k: int = 3,
        vector_field: str = _VECTOR_FIELD,
        select: list[str] = _SELECT_FIELDS,
    ):
        vector_query = RawVectorQuery(vector=embedding_vector, k=k, fields=vector_field)
        return self._search_client.search(  
            search_text=query,  
            vector_queries= [vector_query],
            select=select,
        )

    def upload(
        self,
        documents: list[dict]
    ):
        batch_size = _SearchClient._BATCH_SIZE
        for i in range(0, len(documents), batch_size):
            batch = documents[i:i+batch_size]
            self._search_client.upload_documents(documents=batch)

    @classmethod
    def from_config(cls, config: AzureAiSearchConfig, index_name: str = Constants.INDEX_NAME):
        search_client = SearchClient(config.api_endpoint, index_name, AzureKeyCredential(config.api_key))
        return cls(search_client)

class _SearchIndexClient(object):
    _search_index_client: SearchIndexClient

    def __init__(
        self,
        search_index_client: SearchIndexClient
    ):
        self._search_index_client = search_index_client

    def _get_index_client(self, index_name: str):
        return self._search_index_client.get_index(index_name)

    def _create_index_client(
        self,
        index_name: str,
        with_semantic_search: bool = False
    ):
        fields = [
            SimpleField(name=_ID_FIELD, type=SearchFieldDataType.String, key=True),
            SearchableField(name=_CONTENT_FIELD, type=SearchFieldDataType.String),
            SearchField(name=_VECTOR_FIELD, type=SearchFieldDataType.Collection(SearchFieldDataType.Single), searchable=True, vector_search_dimensions=1536, vector_search_profile=_VECTOR_SEARCH_PROFILE_NAME),
            SearchableField(name=_METADATA_FIELD, type=SearchFieldDataType.String)
        ]

        vector_search = VectorSearch(
            algorithms=[
                HnswVectorSearchAlgorithmConfiguration(
                    name=_HNSW_ALGORITHM_CONFIG_NAME,
                    kind=VectorSearchAlgorithmKind.HNSW,
                    parameters=HnswParameters(
                        m=_M,
                        ef_construction=_EF_CONSTRUCTION,
                        ef_search=_EF_SEARCH,
                        metric=_METRIC
                    )
                )
            ],
            profiles=[
                VectorSearchProfile(
                    name=_VECTOR_SEARCH_PROFILE_NAME,
                    algorithm=_HNSW_ALGORITHM_CONFIG_NAME
                )
            ]  
        )

        semantic_settings: SemanticSettings | None = None
        if with_semantic_search:
            semantic_settings = SemanticSettings(
                configuration=[
                    SemanticConfiguration(
                        name=_DEFAULT_SEMANTIC_CONFIG_NAME,
                        prioritized_fields=PrioritizedFields(
                            prioritized_content_field=[
                                SemanticField(field_name=_CONTENT_FIELD),
                                SemanticField(field_name=_METADATA_FIELD)
                            ]
                        )
                    )
                ]
            )

        index = SearchIndex(
            name=index_name,
            fields=fields,
            vector_search=vector_search,
            semantic_settings=semantic_settings)
        return self._search_index_client.create_index(index)
    
    def get_or_create_index_client(
        self,
        index_name: str,
        with_semantic_search: bool = False
    ):
        try:
            return self._get_index_client(index_name)
        except ResourceNotFoundError:
            print(f"Index {index_name} does not exist... Creating a new index.")
            return self._create_index_client(index_name, with_semantic_search)

    def delete_index(self, index_name: str):
        self._search_index_client.delete_index(index_name)

    @classmethod
    def from_config(cls, config: AzureAiSearchConfig):
        search_index_client = SearchIndexClient(config.api_endpoint, AzureKeyCredential(config.api_key))
        return cls(search_index_client)


class GroundTruthDatasetLoader(object):
    # df keys
    _QUESTION_KEY = "question"
    _ANSWER_KEY = "answer"
    _SOURCE_KEY = "source"
    _SEARCH_RESULT_KEY = "search_result"

    _df: pd.DataFrame

    def __init__(self, path: str = Constants.QA_DATASET_PATH):
        self._df = pd.read_csv(path)

    def load_rows(self):
        for i, row in self._df.iterrows():
            yield i, row

    def get_question_from_row(self, row: pd.Series):
        return row[GroundTruthDatasetLoader._QUESTION_KEY]
    
    def get_answer_from_row(self, row: pd.Series):
        return row[GroundTruthDatasetLoader._ANSWER_KEY]
    
    def get_source_from_row(self, row: pd.Series):
        return row[GroundTruthDatasetLoader._SOURCE_KEY]
    
    def get_seach_answers_from_row(self, row: pd.Series) -> list[SearchResult]:
        search_results = json.loads(row[GroundTruthDatasetLoader._SEARCH_RESULT_KEY])
        return [SearchResult(**search_result) for search_result in search_results]

    def set_search_answers_to_row(self, index: int, search_response: list[SearchResult]):
        value = json.dumps([dataclasses.asdict(elem) for elem in search_response])
        self.set_key(index, GroundTruthDatasetLoader._SEARCH_RESULT_KEY, value)

    def set_key(self, index: int, key: str, value: any):
        self._df.at[index, key] = value

    def save(self, path: str):
        self._df.to_json(path, orient="records")


#### Running the Orchestrator

The following cell runs the end-to-end solution to perform teh following steps:
1. Clears the index if exists.
1. Ingest the embedded chunks into Azure AI Search.
1. Performs Search on the questions in the QA dataset.
1. Evaluates the results of search against the QA dataset.

In [4]:
class ExperimentOrchestrator(object):
    _IN_TOP_KEY = "in_top"
    _ROUGE_L_KEY = "rougeL"

    _dataset: list[dict]
    _index_name: str

    def __init__(
        self,
        search_index_client: _SearchIndexClient,
        search_client: _SearchClient,
        embedding: _Embedding,
        ground_truth_dataset_loader: GroundTruthDatasetLoader,
        index_name: str = Constants.INDEX_NAME,
        embedding_path: str = Constants.EMBEDDING_DATA_PATH
    ):
        # delete the index
        print(f"Deleting index {index_name}...")
        search_index_client.delete_index(index_name)
        time.sleep(2)

        self._search_index_client = search_index_client
        self._search_client = search_client
        self._embedding = embedding
        self._ground_truth_dataset_loader = ground_truth_dataset_loader
        self._index_name = index_name
        self._embedding_data = self._read_json_dataset(embedding_path)

    def _read_json_dataset(self, path: str):
        with open(path, "r") as f:
            return json.load(f)

    def run(self, output_file_path: str):
        # create the index if not exists and uploading the data
        print(f"Creating index and uploading data...")
        self._search_index_client.get_or_create_index_client(self._index_name)
        self._search_client.upload(self._embedding_data)
        time.sleep(2) # adding wait as documents may need to settle

        print("Performing search...")
        for index, row in self._ground_truth_dataset_loader.load_rows():
            question = self._ground_truth_dataset_loader.get_question_from_row(row)
            embedding = self._embedding.embed(question)
            search_result_paged = self._search_client.search(question, embedding)

            search_results_final: list[SearchResult] = []
            for item in search_result_paged:
                del item[_VECTOR_FIELD]
                source = json.loads(item[_METADATA_FIELD])[_SOURCE_METADATA_FIELD]
                search_result = SearchResult(
                    id=item[_ID_FIELD],
                    content=item[_CONTENT_FIELD],
                    score=item["@search.score"],
                    source=source
                )
                search_results_final.append(search_result)
            self._ground_truth_dataset_loader.set_search_answers_to_row(index, search_results_final)

        print("Evaluting chunks...")
        for index, row in self._ground_truth_dataset_loader.load_rows():
            # calculate in top init
            expected_answer = self._ground_truth_dataset_loader.get_answer_from_row(row)
            expected_source = self._ground_truth_dataset_loader.get_source_from_row(row)
            search_results = self._ground_truth_dataset_loader.get_seach_answers_from_row(row)
            search_sources = [search_result.source for search_result in search_results]

            contains_source = 1 if expected_source in search_sources else 0
            self._ground_truth_dataset_loader.set_key(index, ExperimentOrchestrator._IN_TOP_KEY, contains_source)

            # rouge L metric
            max_rougeL_recall = -1
            for search_result in search_results:
                search_content = search_result.content
                rougeL_recall = _rouge_scorer.score(target=expected_answer, prediction=search_content)["rougeL"].recall
                max_rougeL_recall = rougeL_recall if rougeL_recall > max_rougeL_recall else max_rougeL_recall
            self._ground_truth_dataset_loader.set_key(index, ExperimentOrchestrator._ROUGE_L_KEY, max_rougeL_recall)

        self._ground_truth_dataset_loader.save(output_file_path)


    @classmethod
    def from_config(
        cls,
        azure_ai_search_config: AzureAiSearchConfig,
        azure_openai_config: AzureOpenAiConfig,
        index_name: str,
        embedding_path: str
    ):
        search_client_index = _SearchIndexClient.from_config(azure_ai_search_config)
        search_client = _SearchClient.from_config(azure_ai_search_config, index_name)
        embedding = _Embedding.from_config(azure_openai_config)
        
        return cls(search_client_index, search_client, embedding, GroundTruthDatasetLoader(), index_name, embedding_path)

orchestrator = ExperimentOrchestrator.from_config(
    azure_ai_search_config,
    azure_openai_config,
    Constants.INDEX_NAME,
    Constants.EMBEDDING_DATA_PATH
)
orchestrator.run(Constants.OUTPUT_FILE_PATH)

Deleting index evaluation-index...
Creating index and uploading data...
Index evaluation-index does not exist... Creating a new index.
Performing search...
Evaluting chunks...


#### Displaying the Average Evaluation Scores

In [5]:
# reading the output file and getting the average rougeL and in top metric
df = pd.read_json(Constants.OUTPUT_FILE_PATH)
print(f"Average rougeL: {df[ExperimentOrchestrator._ROUGE_L_KEY].mean()}")
print(f"Average in top: {df[ExperimentOrchestrator._IN_TOP_KEY].mean()}")

Average rougeL: 0.953963014495
Average in top: 1.0
